#◢ DeOldify - Colorize your own photos!

####**Credits:**

Special thanks to:

Matt Robinson and María Benavente for pioneering the DeOldify image colab notebook.  

Dana Kelley for doing things, breaking stuff & having an opinion on everything.



---


#◢ Verify Correct Runtime Settings

**<font color='#FF000'> IMPORTANT </font>**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU 


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from os import path
import torch

#◢ Git clone and install DeOldify

In [3]:
!git clone https://github.com/jantic/DeOldify.git DeOldify 

fatal: destination path 'DeOldify' already exists and is not an empty directory.


In [4]:
cd DeOldify

/content/DeOldify


#◢ Setup

In [5]:
!pip install -r requirements.txt

In [0]:
import fastai
from fasterai.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True

In [7]:
!mkdir 'models'
!wget wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth

mkdir: cannot create directory ‘models’: File exists
--2019-05-15 16:17:15--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2019-05-15 16:17:15--  https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth [following]
--2019-05-15 16:17:15--  https://www.dropbox.com/s/raw/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc1e42c22a8dd5907fdd29a1508.dl.dropboxusercontent.com/cd/0/inline/Ag9qZTWacLg20rZbcRXoyFWCsTkjq9zTVt5_4bOBx9anMEDpgIrkIXPwZawGFsO6kKKJ9xv4audCAFZD08cRgj04UBSRNjjsxIoKnF6fr2OJfQ/file# [following]

In [8]:
colorizer = get_image_colorizer(artistic=True)

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:442: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


In [9]:
!pip install ImageHash

### source_url
Type in a url to a direct link of an image.  Usually that means they'll end in .png, .jpg, etc. NOTE: If you want to use your own image, upload it first to a site like Imgur. 

### render_factor
The default value of 35 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the image is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality images in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality images, but the colors may get slightly washed out. 

#### How to Download a Copy
Simply right click on the displayed image and click "Save image as..."!

## Pro Tips
1. You can evaluate how well the image is rendered at each render_factor by using the code at the bottom (that cell under "See how well render_factor values perform on a frame here"). 
2. Keep in mind again that you can go up top and set artistic to False for the colorizer to use the 'Stable' model instead.  This will often tend to do better on portraits, and natural landscapes.  

## Troubleshooting
If you get a 'CUDA out of memory' error, you probably have the render_factor too high.  The max is 45 on 11GB video cards.

#◢ Colorize!!

In [10]:
%%time

import imagehash

def fetch_image(image_url):
  response = requests.get(image_url)
  return PIL.Image.open(BytesIO(response.content)).convert('RGB')

def colorize(orig, render_factor=35):
  origpath = 'orig.png'
  orig.save(origpath)
  colorized = colorizer.get_transformed_image(path=origpath, render_factor=render_factor)
  return colorized


import csv
with open('/content/gdrive/My Drive/SigneBrander/photos.csv') as photoscsv:
  reader = csv.reader(photoscsv)
  for row in reader:
    tweet_id, text, tweet_url, image_url = row
    orig = fetch_image(image_url)
    imghash = imagehash.dhash(orig)
    if str(imghash) == '0000000000000000':
      print("couldn't fetch image {} ({})".format(tweet_id, text))
      continue
    imgpath = '/content/gdrive/My Drive/SigneBrander/colorized/{}.png'.format(imghash)
    if os.path.exists(imgpath):
      print("image {} {} ({}) already found".format(tweet_id, imghash, text))
      continue
    print("colorizing {} {}: {}".format(tweet_id, imghash, text))
    colorized = colorize(orig, 32)
    colorized.save(imgpath)

  

image 895923746757894145 e4bb9162ccccc4d2 (Pyykinkuivatusta rakentamattomalla tontilla, Fabianinkatu 39.- Puutarhakatu 2. - Unioninkatu 40. Etualalla Fabianinkatu (1907, HKM)) already found
image 895969044918095872 249cb8f0dca9b1ef (Näköala Bergan huvila-alueelta (nykyisen Apollonkatu 17:n kohdalta) luoteeseen (1908, Kaupunginmuseo, #helsinki)) already found
image 896014346438012928 d06c6c7aea632109 (Liuksialan kartano (1913 - 1920, Museovirasto)) already found
image 896059645327847425 8894140068ecd4d0 (Eteläesplanadi 14.  Kasarmikatu 48, 46, 44. (1907, Kaupunginmuseo, #helsinki)) already found
image 896240834869428224 d9e9ecf7fbf4f3e9 (Vuorikatu 22, 24 (1907, Kaupunginmuseo, #helsinki)) already found
image 896286133251956736 80102080c3c0a130 (Sauna Marie-Bad (1913, Kaupunginmuseo, #helsinki)) already found
image 896331431907135488 b8e47a8ac4c6c4c8 (Lapinlahdenkatu 9, 7, 5, 3. (1907, Kaupunginmuseo, #helsinki)) already found
image 896376733187923971 f0f890e068c97062 (näkymä Karlbergin 